# Loading, processing, and resaving 2015-2021 data for future use

In [1]:
# required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import openpyxl
import math

import datetime as dt
import matplotlib.dates

print('Done')

Done


In [2]:
# get list of stations
stations_file = '~/Documents/GitHub/HMEI-2021/stations.csv'

df_stations = pd.read_csv(stations_file)
df_stations.head()

,Full,Spaced,Key,Key2,Key3,Key4
0,"Alipur, Delhi - DPCC",Alipur,Alipur,Alipur,Alipur,Alipur
1,"Anand Vihar, Delhi - DPCC",Anand Vihar,Anand,AnandVihar,AnandVihar,AnandVihar
2,"Ashok Vihar, Delhi - DPCC",Ashok Vihar,Ashok,AshokVihar,AshokVihar,AshokVihar
3,"Aya Nagar, Delhi - IMD",Aya Nagar,Aya,AyaNagar,AyaNagar,AyaNagar
4,"Bawana, Delhi - DPCC",Bawana,Bawana,Bawana,Bawana,Bawana


In [3]:
# refer to stations by preset keys
stations20_21 = [s for s in df_stations['Key3'] if type(s) == str]
stations15_19 = [s for s in df_stations['Key4'] if type(s) == str]

In [4]:
print(len(stations20_21))
print(len(stations15_19))

39
33


# Load and join 2015-19 data with 2020-21 data

In [5]:
# check that each date corresponds to the correct index
# valid if nothing is printed
def checkDates(dates, indices, new):
    # if new, check only the "new" stations, those not in the 2015-19 set
    if new:
        stations = [x for x in stations20_21 if x not in stations15_19]
    else:
        stations = stations15_19 # only check stations in the 2015-19 set
    for station in stations:
        printed = False
        if station_data[station].index[indices[0]] != pd.to_datetime(dates[0]):
            print(station)
            printed = True
            print(station_data[station].loc[dates[0]])
        for i in range(1, len(dates)):
            if station_data[station].index[indices[i]] != pd.to_datetime(dates[i]):
                if not printed:
                    print(station)
                    printed = True
                print(station_data[station].loc[dates[i]])
        if printed:
            print()

In [6]:
pollutants = ['PM2.5', 'PM10', 'NO2', 'SO2', 'Ozone']

In [7]:
station_data = {} # dictionary to store each station's data

# load data from Jan, 2015-Nov, 2019
for station in stations15_19:
    file_str = '~/Documents/GitHub/HMEI-2021/Data 2015-19/data_' + station + '.csv'
    df_station = pd.read_csv(file_str, nrows=43345)

    # set datetime as index
    df_station.index = pd.to_datetime(df_station['From Date'], dayfirst=True)

    # reorder columns uniformly
    cols = [p for p in pollutants if p in df_station.columns]
    df_station = df_station[cols]

    end_date = pd.to_datetime('2019-11-30 23:00:00')
    df_station = df_station.loc[:end_date]

    for col in df_station.columns:
        df_station[col] = pd.to_numeric(df_station[col], errors='coerce')

    station_data[station] = df_station
    
    print('done', station)

done Alipur
done AnandVihar
done AshokVihar
done AyaNagar
done Bawana
done CRRI
done DTU
done Dwarka8
done IGI
done ITO
done Jahangirpuri
done JNStadium
done Lodhi_IMD
done MajorDhyanChand
done MandirMarg
done Mundka
done Najafgarh
done Narela
done NehruNagar
done NorthCampus
done NSITDwarka
done Okhla
done Patparganj
done PunjabiBagh
done Pusa_DPCC
done RKPuram
done Rohini
done Shadipur
done Sirifort
done SoniaVihar
done SriAuro
done VivekVihar
done Wazirpur


In [8]:
checkDates(['2015-01-01 00:00:00', '2019-11-30 23:00:00'], [0, -1], False)

In [9]:
# load data for Dec, 2019
for station in stations15_19:
    file_str = '~/Documents/GitHub/HMEI-2021/Data 2015-19/Missing Data/data_' + station + '.xlsx'
    df_station = pd.read_excel(file_str, skiprows=16)

    # set datetime as index
    df_station.index = pd.to_datetime(df_station['From Date'], dayfirst=True)

    # reorder columns uniformly
    cols = [p for p in pollutants if p in df_station.columns]
    df_station = df_station[cols]

    end_date = pd.to_datetime('2019-12-31 23:00:00')
    df_station = df_station.loc[:end_date]

    for col in df_station.columns:
        df_station[col] = pd.to_numeric(df_station[col], errors='coerce')

    station_data[station] = station_data[station].append(df_station) 
    
    print('done', station)

done Alipur
done AnandVihar
done AshokVihar
done AyaNagar
done Bawana
done CRRI
done DTU
done Dwarka8
done IGI
done ITO
done Jahangirpuri
done JNStadium
done Lodhi_IMD
done MajorDhyanChand
done MandirMarg
done Mundka
done Najafgarh
done Narela
done NehruNagar
done NorthCampus
done NSITDwarka
done Okhla
done Patparganj
done PunjabiBagh
done Pusa_DPCC
done RKPuram
done Rohini
done Shadipur
done Sirifort
done SoniaVihar
done SriAuro
done VivekVihar
done Wazirpur


In [10]:
checkDates(['2015-01-01 00:00:00', '2019-12-31 23:00:00'], [0, -1], False)

In [11]:
for station in stations20_21:
    file_str = '~/Documents/GitHub/HMEI-2021/Data 2020-21/data_' + station + '.xlsx'
    df_station = pd.read_excel(file_str, skiprows=16)

    # set datetime as index
    df_station.index = pd.to_datetime(df_station['From Date'], dayfirst=True)

    cols = [p for p in pollutants if p in df_station.columns]
    df_station = df_station[cols]

    end_date = pd.to_datetime('2021-05-31 23:45:00')
    df_station = df_station.loc[:end_date]

    # print(station)
    # print(df_station.loc['2021-05-06 00:15:00'])

    # convert data to numeric
    for col in df_station.columns:
        df_station[col] = pd.to_numeric(df_station[col], errors='coerce')

    # print(df_station.loc['2021-05-06 00:15:00'])

    hourly = df_station.resample('H').mean().loc[:'2021-05-31']

    # print(hourly.loc['2021-05-06 01:00:00'])
    # print()

    if station in stations15_19:
        station_data[station] = station_data[station].append(hourly)
    else:
        station_data[station] = hourly
    
    print('done', station)

done Alipur
done AnandVihar
done AshokVihar
done AyaNagar
done Bawana
done Chandni
done CRRI
done DTU
done Dwarka8
done EastArjun
done IGI
done IHBAS
done ITO
done Jahangirpuri
done JNStadium
done Karni
done Lodhi_IITM
done Lodhi_IMD
done MajorDhyanChand
done MandirMarg
done Mundka
done Najafgarh
done Narela
done NehruNagar
done NorthCampus
done NSITDwarka
done Okhla
done Patparganj
done PunjabiBagh
done Pusa_DPCC
done Pusa_IMD
done RKPuram
done Rohini
done Shadipur
done Sirifort
done SoniaVihar
done SriAuro
done VivekVihar
done Wazirpur


In [12]:
checkDates(['2015-01-01 00:00:00', '2021-05-31 23:00:00', '2019-12-31 23:00:00', '2020-01-01 00:00:00'], 
           [0, -1, 43823, 43824], False)
checkDates(['2020-01-01 00:00:00', '2021-05-31 23:00:00'], [0, -1], True)

In [13]:
for station in station_data:
    print(station_data[station].shape)

(56232, 5)
(56232, 5)
(56232, 5)
(56232, 4)
(56232, 5)
(56232, 4)
(56232, 5)
(56232, 5)
(56232, 4)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 4)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 4)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(56232, 5)
(12408, 5)
(12408, 2)
(12408, 5)
(12408, 5)
(12408, 5)
(12408, 4)


# Clean data

In [14]:
# remove values which repeat for >= 24 consecutive hours
# and values in excess of the monitor's range (> 999.99)
def removeBadVals(df, station):
    printed = False
    ts_index = df.index
    
    for col in df.columns:
        col_printed = False
        
        i = 0
        while i + 24 < len(df[col]):
            val = df.loc[ts_index[i], col]
            
            # if value is missing
            if math.isnan(val):
                i += 1
                continue
                
            # if value is above the reported range
            if val > 999.99:
                df.loc[ts_index[i], col] = math.nan
                if not printed:
                    print(station)
                    print()
                    printed = True
                if not col_printed:
                    print(col + ':')
                    col_printed = True
                print(str(ts_index[i]) + '\t\t\t\tExcess Value: ' + str(val))
                print()
                i += 1
                continue
                
            at_24 = df.loc[ts_index[i+24], col]
            
            # if values 24-hours apart do not match
            if math.isnan(at_24):
                i += 1
                continue
            if val != at_24:
                i += 1
                continue
            
            # if values 24-hours apart *do* match
            j = i
            k = i + 24
            delete = True
            while k >= j:
                at_j = df.loc[ts_index[j], col]
                at_k = df.loc[ts_index[k], col]
                if math.isnan(at_j) or math.isnan(at_k):
                    delete = False
                    break
                # check that all values between indices j and k are the same
                elif at_j != at_k:
                    delete = False
                    break
                # and that they are the same as the initial value at index i
                elif at_j != val:
                    delete = False
                    break
                j += 1
                k -= 1
                
            # replace all repeated values with NaN, except the first
            if delete:
                end = i + 24
                while (end < len(df[col])) and (not math.isnan(df.loc[ts_index[end], col])) and \
                      (val == df.loc[ts_index[end], col]):
                    end += 1
                for index in range(i+1, end):
                    df.loc[ts_index[index], col] = math.nan
                if not printed:
                    print(station)
                    print()
                    printed = True
                if not col_printed:
                    print(col + ':')
                    col_printed = True
                print(str(ts_index[i]) + ' | ' + str(ts_index[end-1]) + '\tValue: ' + str(val))
                print()
                i = end
                
            else:
                i += 1
                
        # check remaining values to see if they exceed the reported range
        while i < len(df[col]):
            val = df.loc[ts_index[i], col]
            if val > 999.99:
                df.loc[ts_index[i], col] = math.nan
                if not printed:
                    print(station)
                    print()
                    printed = True
                if not col_printed:
                    print(col + ':')
                    col_printed = True   
                print(str(ts_index[i]) + '\t\t\t\tExcess Value: ' + str(val))
                print()
            i += 1
            
    if printed:
        print()

In [15]:
for station in station_data:
    removeBadVals(station_data[station], station)

AnandVihar

PM2.5:
2019-03-19 09:00:00 | 2019-03-20 10:00:00	Value: 182.0

PM10:
2016-09-04 17:00:00				Excess Value: 1000.0

2017-12-31 20:00:00				Excess Value: 1000.0

2018-02-09 00:00:00				Excess Value: 1000.0

2018-02-09 01:00:00				Excess Value: 1000.0

2019-06-11 05:00:00				Excess Value: 1000.0


AyaNagar

PM2.5:
2018-03-13 16:00:00 | 2018-03-16 12:00:00	Value: 8.0

2018-03-22 19:00:00 | 2018-03-27 18:00:00	Value: 8.0

2018-10-28 02:00:00 | 2018-10-29 11:00:00	Value: 10.0

2018-11-02 05:00:00 | 2018-11-03 15:00:00	Value: 195.24

NO2:
2019-07-22 14:00:00 | 2019-07-25 08:00:00	Value: 63.83


CRRI

PM10:
2018-12-04 00:00:00				Excess Value: 1000.0

NO2:
2018-02-14 19:00:00 | 2018-02-16 14:00:00	Value: 24.0

2018-03-17 17:00:00 | 2018-03-20 23:00:00	Value: 24.0

2021-03-31 12:00:00 | 2021-04-05 14:00:00	Value: 178.1

2021-04-19 02:00:00 | 2021-05-31 23:00:00	Value: 178.1

Ozone:
2018-02-14 19:00:00 | 2018-02-16 14:00:00	Value: 10.0

2018-03-17 17:00:00 | 2018-03-20 23:00:00	Value:

2017-05-02 15:00:00				Excess Value: 1000.0

2017-10-20 01:00:00				Excess Value: 1000.0

2017-10-20 02:00:00				Excess Value: 1000.0

2017-10-20 03:00:00				Excess Value: 1000.0

2017-10-20 04:00:00				Excess Value: 1000.0

2017-10-20 05:00:00				Excess Value: 1000.0

2017-10-20 08:00:00				Excess Value: 1000.0

2017-10-20 09:00:00				Excess Value: 1000.0

2017-10-20 10:00:00				Excess Value: 1000.0

2017-11-10 18:00:00				Excess Value: 1000.0

2018-01-12 16:00:00				Excess Value: 1000.0

2018-01-31 12:00:00				Excess Value: 1000.0

2018-03-21 16:00:00				Excess Value: 1000.0

2018-03-23 17:00:00				Excess Value: 1000.0

2018-03-31 12:00:00				Excess Value: 1000.0

2018-04-06 17:00:00				Excess Value: 1000.0

2018-04-06 18:00:00				Excess Value: 1000.0

2018-04-16 12:00:00				Excess Value: 1000.0

2018-04-17 17:00:00				Excess Value: 1000.0

2018-05-02 12:00:00				Excess Value: 1000.0

2018-05-16 16:00:00				Excess Value: 1000.0

2018-05-16 18:00:00				Excess Value: 1000.0

2018-05-28

# Resave data for future use

In [16]:
for station in station_data:
    station_data[station].to_csv('new_station_data/data_' + station + '.csv')